# Binaryzacja

### Zadanie domowe - binaryzacja adaptacyjna w oknach z interpolacją.

Pokazana w ramach podstawowej części ćwiczenia binaryzacja adaptacyjna działa dobrze, ale jest dość złożona obliczeniowo (choć oczywiście należy mieć świadomość, że implementację można zoptymalizować i wyeliminować pewne powtarzające się obliczenia).
Zbliżone rozwiązanie można również realizować w nieco innym wariancie - w oknach.
Ogólna idea jest następująca: wejściowy obraz dzielimy na nienachodzące (rozłączne) okna - wygodnie jest założyć, że są one kwadratowe i o rozmiarze będącym potęgą liczby 2.
W każdym z okien obliczamy próg - niech to będzie średnia i stosujemy do binaryzacji lokalnej.
Jak nietrudno się domyślić efekt nie będzie dobry, gdyż na granicach okien wystąpią artefakty.
Aby je wyeliminować należy zastosować interpolację, co zostanie szczegółowo opisane poniżej.
Warto zaznaczyć, że podobny mechanizm interpolacji stosowany jest w poznanym wcześniej algorytmie CLAHE.
Zainteresowane osoby odsyłam do artykułu na [Wikipedii](https://en.wikipedia.org/wiki/Adaptive_histogram_equalization) oraz do artykułu o metodzie CLAHE - Zuiderveld, Karel. “Contrast Limited Adaptive Histograph Equalization.” Graphic Gems IV. San Diego: Academic Press Professional, 1994. 474–485.



Na początek zaimplementujemy wariant metody bez interpolacji:
1. Wczytaj obraz _rice.png_.
2. W dwóch pętlach `for`, dla okien o ustalonym wymiarze $W$ (potęga 2), oblicz średnią:
- pętle powinny mieć krok $W$,
- wynik (tj. średnie) należy zapisać w pomocniczej tablicy,
- przydatny operator to `//` - dzielenie całkowitoliczbowe (*floor division*).

3. W kolejnych dwóch pętlach `for` (tym razem o kroku 1) przeprowadź binaryzację z wyznaczonymi progami.
   Tu oczywiście należy się sprytnie odwołać do wyników z tablicy pomocniczej.
   Wyświetl wyniki - czy jest on poprawny?
   Podpowiedź - błędy lepiej widać dla mniejszego rozmiaru okna (np. 16 x 16).

In [ ]:
# ROZWIĄZANIE ZADANIA NIŻEJ

4. Rozwiązaniem problemu artefaktów na obrazie jest zastosowanie interpolacji.
   Próg binaryzacji dla danego okna wyliczany jest na podstawie progów z sąsiednich okien.
   ![Ilustracja koncepcji interpolacji](https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/clahe_tile_interpolation.png)

   Koncepcja została przedstawiona na powyższym rysunku.
   Możliwe są 3 przypadki:
   - piksel leży w rogach obrazu (kolor czerwony) - wtedy za próg przyjmuje się wartość średniej obliczonej dla danego okna,
   - piksel leży na krawędzi obrazu (kolor zielony) - wtedy za próg przyjmuje się wartość obliczoną na podstawie średnich z dwóch sąsiednich okien,
   - piksel leży w środku (kolor fioletowy) - wtedy próg jest obliczany na podstawie 4 sąsiednich okien.

   Uwaga. Proszę zwrócić uwagę, że sprawa jest dość złożona.
   Obraz dzielimy na okna (dla nich liczymy średnią) i następnie każde z okien "wirtualnie" na cztery sub-okna (linie przerywane).
   To ułatwia znalezienie środków okien (czarne kwadraty), które są wykorzystywane w interpolacji.

5. Implementujemy interpolację.
   Potrzebujemy do tego znać progi (jeden, dwa lub cztery), ale dla przejrzystości obliczeń lepiej zawsze przyjąć cztery oraz odległości od rozważnego piksela do środka sąsiednich okien (też w ogólnym przypadku 4):
   - całość sprowadza się do określania pozycji piksela,
   - na początek rozważmy przypadek czterech narożników (kolor czerwony na rysunku) - trzeba napisać `if`, który je wyznaczy,
   - warto sprawdzić, czy nie popełniliśmy błędu i np. tymczasowo do obrazu wynikowego w tym miejscu przypisać wartość 255. Efekt powinien być taki, że widoczne będą tylko narożniki.
   - drugi przypadek do brzegi (kolor zielony) - postępujemy podobnie jak przy narożnikach, przy czym osobno wydzielamy brzegi pionowe i poziome. Tu też warto sobie obrazek "pokolorować".
   - na koniec wyznaczamy piksele w środku.
   - analizując poprawność proszę zwrócić uwagę na to, żeby nie było przerw pomiędzy obszarami.
   - mając podział możemy dla każdego z obszarów wyliczyć cztery progi ($t11, t12, t21, t22$):
        - dla narożników wartość ta będzie identyczna i wynosi po prostu `t11 =t[jT][iT]`, gdzie `iT=i//W` oraz `jT=j//W`.
          Uwaga. Proszę używać indeksów tymczasowych $jT,iT$, gdyż będą potrzebne w dalszych obliczeniach.
        - dla brzegów pionowych występują dwie wartości: okno bieżące i sąsiednie.
          Wyznaczenie współrzędnej poziomej jest proste (jak dla narożników).
          Nad współrzędną pionową trzeba się chwilę zastanowić - aby nie rozważać wielu przypadków można od bieżącej współrzędnej odjąć połowę rozmiaru okna i dopiero później wykonać dzielenie przez rozmiar okna.
          W ten sposób otrzymujemy indeks okna o mniejszej współrzędnej.
          Indeks drugiego uzyskamy dodając 1.
          Proszę się zastanowić dlaczego to działa - najlepiej to sobie rozrysować.
        - dla brzegów poziomych należy postąpić analogicznie,
        - obliczenia dla obszaru wewnątrz powinny być już oczywiste.
   - kolejny krok to wyliczenie odległości pomiędzy rozważanym pikselem, a czterema środkami.
     Przykładowo dla osi X wygląda to następująco: `dX1 = i - W/2 - iT*W` oraz `dX2 = (iT+1)*W - i-W/2`.
     Dla osi Y analogicznie.
     Ponownie proszę się zastanowić dlaczego to jest poprawne - najlepiej to sobie narysować.
   - ostatni krok to interpolacja dwuliniowa.
     Wykonamy ją w trzech krokach:
     - interpolacja w osi X dla dwóch górnych okien - sprowadza się ona do średniej ważonej pomiędzy wartościami $t11$ i $t12$, przy czym wagi to odpowiednio $dX2/W$ i $dX1/W$.
       Ponownie na podstawie rysunku proszę to przemyśleć.
     - interpolacja w osi X dla dolnych okien jest analogiczna,
     - interpolacja w osi Y również jest analogiczna, z tym, że wejściem są dwa wyniki interpolacji w poziomie.

6. "Kropka nad i" to oczywiście binaryzacja z wyznaczonym poprzez interpolację progiem - proszę dobrać rozmiar okna.
7. Na koniec proszę porównać na wspólnym rysunku poznane metody binaryzacji:
- Otsu,
- lokalna na podstawie średniej,
- lokalna Sauvoli,
- lokalna w oknach bez interpolacji,
- lokalna w oknach z interpolacją.

Proszę pod porównaniem, w osobnej sekcji *markdown*, krótko skomentować uzyskane wyniki.

In [ ]:

import matplotlib.pyplot as plt
import cv2
import numpy as np
import os

if not os.path.exists("rice.png") :
    !wget https://raw.githubusercontent.com/vision-agh/poc_sw/master/04_Thresholding/rice.png --no-check-certificate
rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)


def BIN_IN_WIN_WITH_INT(image):
    X, Y = image.shape
    W = 16
    image_without_int = np.zeros(image.shape)
    thre=np.zeros((X//W,Y//W))

    for i in range(0, X, W):
        for j in range (0, Y, W):
            place  = image[i:i+W,j:j+W]
            thre[i//W][j//W] = np.mean(place)

            
    for i in range(X):
        for j in range(Y):
            image_without_int[i][j]=(image[i][j]>thre[i//W][j//W]).astype('uint8')
            
    image_with_int = np.zeros(np.shape(image))
    inter_bilinear = np.zeros(np.shape(image))

    w = W//2
    for i in range(X):
        for j in range (Y):
            iT, jT = i // W, j // W
            if (i <= w and j <= w) or (i >= Y - w and j <= w)  or (i <= w and j >= X - w) or (i >= Y - w and j >= X - w):
                t11 = thre[iT][jT]
                t12 = thre[iT][jT]
                t21 = thre[iT][jT]
                t22 = thre[iT][jT]
            elif (i <= w or i >= Y - w) and (j >= w and j <= X - w):
                t11 = thre[iT][((j - w) // W)]
                t21 = thre[iT][((j - w) // W)]
                t12 = thre[iT][1 + (j - w) // W]
                t22 = thre[iT][1 + (j - w) // W]
            elif (  i>= w and i <= X-w) and (j<=w or j >= X-w): 
                t11 = thre[(i - w) // W][jT]
                t21 = thre[(i - w) // W][jT]
                t12 = thre[1 + (i - w) // W][jT]
                t22 = thre[1 + (i - w) // W][jT]
            else:
                t11 = thre[iT][(j - w) // W]
                t21 = thre[(i - w) // W][jT]
                t12 = thre[iT][1 + (j - w) // W]
                t22 = thre[1 + (i - w) // W][jT]

            dX1 = np.abs(j - w - jT * W)
            dX2 = np.abs((jT+1) * W - j - w)
            dY1 = np.abs(i - w - iT * W)
            dY2 = np.abs((iT+1) * W - i - w)

            licz_inter = (t11 * (dX2 / W) + t12 * (dX1 / W) + t21 * (dY2 / W) + t22 * (dY1 / W))
            mian_inter = (dX2 / W + dX1 / W + dY2 / W + dY1 / W)
            inter_bilinear[i][j] = licz_inter/mian_inter

    for i in range(0, X):
        for j in range(0, Y):
            image_with_int[i][j] = image[i][j] > inter_bilinear[i][j]

    return image_without_int, image_with_int



def loc_bin_suv_minus(image, size =16, R=128,k=0.15):
   
    (X,Y) = image.shape
    new_image = np.zeros(image.shape)
    W = size
    w = int(W/2)
    for j in range(w, Y-w):
        for i in range(w, X-w):
            plane = image[(i-w):(i+w),(j-w):(j+w)]
            mean = np.mean(plane)*(1 - k*(np.std(plane)/R - 1))
            new_image[i][j] = image[i][j] > mean
    return new_image


def OTSU(image):
    H = cv2.calcHist([image],[0], None, [256],[0,256])
    
    H_cs = H.cumsum()
    H = np.squeeze(H)
    H_norm=(H/(image.shape[0]*image.shape[1]))
    histogram_cumu_normalized=(H_cs/(image.shape[0]*image.shape[1]))

    m_g = 0
    for i in range(np.size(H_norm)):
        m_g += i*H_norm[i]
    
    sigma_b = np.zeros(256)
    
    for k in range(256):
        P0_k = histogram_cumu_normalized[k]
        
        if P0_k>0 and P0_k<1:
            m_k = np.dot(np.arange(k+1),H_norm[0:k+1])
            sigma_k_licznik = (m_g*P0_k-m_k)**2
            sigma_k_mianownik = P0_k*(1-P0_k)
            sigma_b[k]= sigma_k_licznik/sigma_k_mianownik

    
    maximum = max(sigma_b)
    for i in range(np.size(sigma_b)):
        if sigma_b[i] == maximum:
            thre = i
    
    plt.rcParams["figure.figsize"] = (10,5)
    return thre, sigma_b




def binaryzacja(image,thre):
    for i in range(np.shape(image)[0]):
        for j in range(np.shape(image)[1]):
            if image[i][j] < thre:
                image[i][j] = 0
            else:
                image[i][j] = 1
    return image



def loc_bin(image, size):
   
    (X,Y) = image.shape
    new_image = np.zeros(image.shape)
    W = size
    w = int(W/2)
    for j in range(w, Y-w):
        for i in range(w, X-w):
            plane = image[(i-w):(i+w),(j-w):(j+w)]
            mean = np.mean(plane)
            new_image[i][j] = image[i][j] > mean
    return new_image


plt.imshow(rice,'gray')
plt.axis('off')
plt.title('Obraz oryginalny')
plt.show()

plt.imshow(BIN_IN_WIN_WITH_INT(rice)[0],'gray')
plt.axis('off')
plt.title('Binaryzacja bez interpolacji')
plt.show()


plt.imshow(BIN_IN_WIN_WITH_INT(rice)[1],'gray')
plt.axis('off')
plt.title('Binearyzacja z interpolacją')
plt.show()

plt.imshow(binaryzacja(rice, OTSU(rice)[0]),'gray')
plt.axis('off')
plt.title('Binezeryzacja metodą OTSU')
plt.show()


rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(loc_bin_suv_minus(rice),'gray')
plt.axis('off')
plt.title('Binearyzacja metodą Sauvoli')
plt.show()

rice = cv2.imread("rice.png", cv2.IMREAD_GRAYSCALE)
plt.imshow(loc_bin(rice,16),'gray')
plt.axis('off')
plt.title('Binearyzacja metodą średniej')
plt.show()



Najlepszy rezultat dała metoda Sauvoli - najlepiej zachowane tło i szczegoły ( pojawiła się czarna ramka). Na obrazie powstałym metodą OTSU  można zaobserwować  brak dokładności, niektóre elementy są niewidoczne - brak szczegółów, bądź elementy są zniekształcone. Na obrazie uzyskanym metodą średniej dobrze widać szczegóły ( kształty),tło nie jest najlpiej zachowan, pojawiła się czarna ramka. Binearyzacja z interpolacją dała dobry efekt, plasuje się za Sauvolim.